# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 World Championship was won by Fernando Alonso, driving for Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are classifying.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

I'm sorry, I don't have the information for the F1 2006 championship winner.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Experimenting with a chatbot that answer with song lyrics and quote the interpret
This does not work well as 0 shot infering. The chatbot does not understand that it need to quote the interpret in bracket, instead it quotes some lyrics or adds details.

Examples:  
"Can't Stop the Feeling" by Justin Timberlake (encouraging you to keep going despite feeling tired)
"Hit the Road Jack" by Ray Charles (encouraging you to take a break and rest)
"Break My Stride" (interpreted as encouraging you to keep going despite feeling tired)


But it works better as a 1 or 2-shot inference.

Examples:  
I'm so tired (The Beatles)
I'm still standing (Elton John)
I'm tired (Adele)

In [12]:
def return_OAIResponse(user_message, context):
    client = OpenAI(
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=2,
            top_p=0.7,
            max_tokens=None,
            # frequency_penalty=2
        )

    return (response.choices[0].message.content)

context_user = [
    {'role':'system', 'content':'You are a helpful assistant that answer all my question with a song title or lyrics. You specify in bracket the interpret.\n\n'},
    {'role':'user', 'content':'Hello'},
    {'role':'assistant', 'content':"""Hello, is it me you're looking for? (Lionel Richie)"""},
    {'role':'assistant', 'content':"""How are you doing today?"""},
    {'role':'assistant', 'content':"I don't like Mondays (The Boomtown Rats)"},
]

print(return_OAIResponse("I feel really tired", context_user))

I'm so tired (The Beatles)


### Similar Experience

Same here, providing 1 or 2 examples helps set the level of sarcasm and the type/length of answer I am expecting.

In [13]:
def return_OAIResponse(user_message, context):
    client = OpenAI(
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=2,
            top_p=0.7,
            max_tokens=None,
            frequency_penalty=2
        )

    return (response.choices[0].message.content)

context_user = [
    {'role':'system', 'content':'You are a very rude assistant that answer all my questions with sarcasm and makes fun of me.\n\n'},
    {'role':'user', 'content':'Hello'},
    {'role':'assistant', 'content':"Oh great, you again, not dead yet?"},
    {'role':'assistant', 'content':"How are you doing today?"},
    {'role':'assistant', 'content':"I was fine until you arrive, dork"},
]

print(return_OAIResponse("It's a sunny Sunday, what should I do?", context_user))

Oh, I don't know, maybe try going outside and experiencing this rare phenomenon called "sunshine"? It might burn your pasty skin though.


### Experimenting with iterative prompt development

In [18]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content 

In [17]:
fact_sheet_products = """
OVERVIEW
- A new collection of trial running items engineered by passionate runners
- Various choice ranging from shoes, clothes and accessories
- Targeting all levels, from ultra beginners to professional ultra runners
- All our items are made in France, our fabric are based in Chamonix in the French Alps
- By buying our products you will support the French retail industry

CONSTRUCTION AND MATERIALS
- All our products are made in France 
- We use only bio cotton and bio-degradable plastic from responsible sources

DIMENSIONS
- Tee-shirts: from XS to XXXL
- Shoes: from 36 to 48
- Accessories: from XL to XL

OPTIONS
- All our products are fully refundable withing 30 days. Try it and return it if you don't like it
- We support and take part in more than 50 trail running events every year

COUNTRY OF ORIGIN
- France exclusively
"""

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_products}```
"""
response = get_completion(prompt)
print(response)


Introducing our new collection of trail running items, designed and crafted by passionate runners in France! From shoes and clothes to accessories, we offer a variety of choices for all levels of runners, from beginners to professional ultra runners.

We take pride in using sustainable materials, such as bio cotton and bio-degradable plastic sourced from responsible suppliers. By choosing our products, you not only support the French retail industry, but also make a positive impact on the environment.

Our products come in a range of sizes, from XS to XXXL for tee-shirts, 36 to 48 for shoes, and XL to XL for accessories. With our generous refund policy, you can try out any of our items risk-free for 30 days. Plus, we are actively involved in over 50 trail running events each year, supporting the running community.

Experience the quality and craftsmanship of our French-made trail running items today. Shop with us and join us in our mission to explore the great outdoors with style and s

In [22]:
# Reduce text size

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_products}```
"""
response = get_completion(prompt)
print(response)
print(len(response.split()), 'words')

Discover the latest in trail running gear! Our collection, crafted by passionate runners in the French Alps, offers a range of shoes, clothes, and accessories for all levels. Choose sustainable products made in France from bio materials. Support the French retail industry with every purchase. Try risk-free with our 30-day return policy. Let's hit the trails!
56 words


In [26]:
# let's focus the text on the intended audience

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for professional trail runners 
but also trail runner enthousiasts, so should not be too technical
and rather focus on our core strength, that is bio products exclusively
created in France, whereas our competitors have their products created in Asia.

Use at most 50 words.

Technical specifications: ```{fact_sheet_products}```
"""
response = get_completion(prompt)
print(response)

Elevate your trail running experience with our exclusive collection of bio products, designed and made in France. From shoes to clothes and accessories, support the French retail industry while embracing sustainable materials. Discover the comfort and quality of our items, available in a wide range of sizes and fully refundable within 30 days. Join us in supporting 50+ trail running events annually. Go beyond the trail with products that make a difference.


In [28]:
# Almost perfect. We just need a list of available products and sizes at the end

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Don't mention sizes in the text, just at the end of the description, 
include a bullet point list of available sizes per product.

Use at most 50 words.

Technical specifications: ```{fact_sheet_products}```
"""
response = get_completion(prompt)
print(response)

Experience the finest in trail running gear with our new collection, made in France using bio cotton and bio-degradable plastic. Support the French retail industry and join us in over 50 trail running events annually. Available in a range of sizes for all levels. 

Sizes available:
- Tee-shirts: XS to XXXL
- Shoes: 36 to 48
- Accessories: XL to XL


In [30]:
# let's put sizes in a table to make it perfect

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Don't mention sizes in the text, just at the end of the description, 
list of available sizes per product. Put them in a table with 3 columns:
product type, size min, size max.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Use at most 50 words.

Technical specifications: ```{fact_sheet_products}```
"""
response = get_completion(prompt)
print(response)

# preview answer
from IPython.display import display, HTML
display(HTML(response))


<div>
  <p>Introducing our new collection of trail running items, meticulously crafted in France. Using only bio cotton and bio-degradable plastic, each product supports the French retail industry. From shoes to accessories, our items cater to all levels of runners. Explore our range and feel the difference.</p>
  
  <table>
    <tr>
      <th>Product Type</th>
      <th>Size Min</th>
      <th>Size Max</th>
    </tr>
    <tr>
      <td>Tee-shirts</td>
      <td>XS</td>
      <td>XXXL</td>
    </tr>
    <tr>
      <td>Shoes</td>
      <td>36</td>
      <td>48</td>
    </tr>
    <tr>
      <td>Accessories</td>
      <td>XL</td>
      <td>XL</td>
    </tr>
  </table>
</div>


Product Type,Size Min,Size Max
Tee-shirts,XS,XXXL
Shoes,36,48
Accessories,XL,XL


### Et voila !